In [1]:
# imports

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Optional

from einops import rearrange
from einops.layers.torch import Rearrange
from matplotlib.gridspec import GridSpec
from matplotlib.colors import CenteredNorm
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

from commons import DEVICE, DATA_PATH, RUNS_PATH, Array, Float, Int
from data import FancyDataset
from influence import get_influences
from models import make_mlp

In [2]:
# run configuration

RUN_NAME: str = "MNIST-10-epochs"

# MNIST dataset
IN_FEATURES: int = 28 * 28
OUT_FEATURES: int = 10

# model architecture
HIDDEN_FEATURES: int = 128
NUM_HIDDEN_LAYERS: int = 4

BATCH_SIZE: int = 32

In [3]:
def get_training_states(run_name: str = RUN_NAME) -> list:
    files = [f for f in (RUNS_PATH / run_name).iterdir()]
    sorted_files = sorted(files, key=lambda f: int(f.stem[:2]))
    
    models = []
    batch_indices = []

    for file in sorted_files:
        file_type = file.stem[3:]

        if file_type == "state_dict":
            mlp = make_mlp(
                in_features=IN_FEATURES,
                out_features=OUT_FEATURES,
                hidden_features=HIDDEN_FEATURES,
                num_hidden_layers=NUM_HIDDEN_LAYERS,
            )
            state_dict = torch.load(file)
            mlp.load_state_dict(state_dict)
            model = nn.Sequential(
                Rearrange("b c h w -> b (c h w)"),
                mlp,
            )
            models.append(model)
        elif file_type == "indices":
            indices = torch.load(file).to("cpu")
            batch_indices.append(indices)

    return list(zip(models, batch_indices))

training_states = get_training_states()

In [4]:
train_set = FancyDataset(
    MNIST(
        root=DATA_PATH,
        train=True,
        transform=ToTensor(),
        download=True,
    )
)
test_set = FancyDataset(
    MNIST(
        root=f"{DATA_PATH}",
        train=False,
        transform=ToTensor(),
        download=True,
    )
)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
def get_losses():
    best_model, _ = training_states[-1]
    best_model = best_model.to(DEVICE)

    losses = torch.zeros(len(train_set))
    for indices, inputs, targets in train_loader:
        inputs = inputs.to(DEVICE)
        targets = targets.to(DEVICE)
        outputs = best_model(inputs)
        losses[indices] = F.cross_entropy(outputs, targets, reduction="none").to("cpu")

    return losses

losses = get_losses()

In [6]:
highest_loss_indices = torch.argsort(losses, descending=True)[:128]

In [7]:
catalyst_set = reactant_set = Subset(train_set, highest_loss_indices)
catalyst_loader = reactant_loader = DataLoader(
    reactant_set,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

In [8]:
influences = get_influences(
    training_states=training_states,
    catalyst_loader=catalyst_loader,
    reactant_loader=reactant_loader,
    device=DEVICE,
)
influences = influences.detach().to("cpu")

In [9]:
def get_top_k_catalysts_indices(influences: Float[Array, "c r"], k: int = 128):
    catalyst_magnitudes = torch.einsum("c r -> c", influences ** 2)
    top_k_catalysts = torch.argsort(catalyst_magnitudes, descending=True)[:k]
    return top_k_catalysts

In [10]:
def visualize_influence_images(
    catalyst_indices: Int[Array, "c"],  # into full training set
    reactant_indices: Int[Array, "r"],  # into full test set
    influences: Float[Array, "c r"],
    k: Optional[int] = None,
):
    height, width, channels = 28, 28, 1

    if k is not None:
        num_catalysts = k
        num_reactants = k
        catalyst_indices = catalyst_indices[:k]
        reactant_indices = reactant_indices[:k]
        influences = influences[:k, :k]
    else:
        num_catalysts = len(catalyst_indices)
        num_reactants = len(reactant_indices)

    fig = plt.figure(figsize=(num_reactants + 1, num_catalysts + 1))
    gs = GridSpec(
        nrows=(num_catalysts + 1),
        ncols=(num_reactants + 1),
        width_ratios=([1] * num_reactants + [0.05]),
        wspace=0.5,
        hspace=0.5,
    )

    for index_into_catalyst, index_into_train in enumerate(catalyst_indices):
        index_into_train = index_into_train.item()
        _, catalyst_image, catalyst_label = train_set[index_into_train]
        catalyst_image = rearrange(catalyst_image, "k h w -> h w k")
        ax = fig.add_subplot(gs[index_into_catalyst + 1, 0])
        ax.imshow(catalyst_image, cmap="gray")
        ax.text(-5, width // 2, str(catalyst_label), va='center', ha='right')
        ax.axis("off")

        if index_into_catalyst == 0:  # only label the first one to avoid redundancy
            ax.set_title("Catalysts", va='bottom', ha='center')


    for index_into_reactant, index_into_train in enumerate(reactant_indices):
        index_into_train = index_into_train.item()
        _, reactant_image, reactant_label = train_set[index_into_train]
        reactant_image = rearrange(reactant_image, "k h w -> h w k")
        ax = fig.add_subplot(gs[0, index_into_reactant + 1])
        ax.imshow(reactant_image, cmap="gray")
        ax.text(height // 2, -5, str(reactant_label), va='top', ha='center')
        ax.axis("off")

    ax = fig.add_subplot(gs[1:, 1:-1])
    cax = ax.matshow(influences, cmap="bwr", norm=CenteredNorm())
    ax.axis("off")

    plt.show()


In [12]:
visualize_influence_images(
    catalyst_indices=highest_loss_indices,
    reactant_indices=highest_loss_indices,
    influences=influences,
    k=128,
)